<h1>Inverse Kinematics</h1>
<h3>Interactive Demo below</h3>

Based on <a href="https://ipvs.informatik.uni-stuttgart.de/mlr/marc/teaching/index.html">Marc Toussaint's robotics lecture</a>.

In [6]:
%matplotlib inline
import numpy as np
import numpy.linalg
import numpy.random
import pandas as pd
import matplotlib.pyplot as py
import threading
import IPython.display as disp
from ipywidgets import widgets

kinematics differ from dynamics in that they control the generalized coordinates $q$ directly, instead of generalized accelerations $\ddot{q}$.

<h2>Kinematic Chain</h2>

A simple model is the kinematic chain (subset of a kinematic tree) where each joint is attached to the previous joint.
In this example I'm using simple rotational joints with two DoFs: the angle and extension $q = (\alpha,\ l)$. This is formulated using affine transformations: <br>
<br>
$\begin{align}
A(\alpha,\ l) &= R(\alpha)\ T(l) \\
\\
R(\alpha) &= \left\lgroup\matrix{
\cos(\alpha) & -\sin(\alpha) & 0 \\
\sin(\alpha) &  \cos(\alpha) & 0 \\
0 & 0 & 1}\right\rgroup \\
\\
T(l) &= \left\lgroup\matrix{
1 & 0 & l \\
0 & 1 & 0 \\
0 & 0 & 1}\right\rgroup
\end{align}$

This could be multiplied out but for readability in the code I'll keep them seperated. <br>
<br>
The chain <b>TODO:</b>

In [7]:
# Simple rotational joint
def linkTransformation(angle, length):
    R = np.matrix([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0,              0,             1],
    ])
    
    T = np.matrix([
        [1, 0, length],
        [0, 1, 0],
        [0, 0, 1],
    ])
    
    return R * T
    
# Returns a set of transformations for a chain of joints
# attached to one another.
def kinematicChain(angles, lenghts):
    assert(len(angles) == len(lenghts))
    A = np.matrix(np.eye(3))
    trafos = np.zeros((len(angles), 3, 3))
    for i in range(0, len(angles)):
        linkA = linkTransformation(angles[i], lenghts[i])
        A = np.dot(A, linkA)
        trafos[i, :, :] = A
    return trafos

def effectorPositions(transformations):
    effPositions = np.zeros((len(transformations) + 1, 2))
    for i in range(0, len(transformations)):
        A = transformations[i]
        effPositions[i + 1, :] =  np.dot(A, [0, 0, 1])[:-1]
    return effPositions

<h2>An iterative IK Solver</h2>
<br>
Linearization of the transformation function: <br>
$\begin{align}
\phi(q - q_0) &\approx y_0 + \frac{\partial T}{\partial q} (q - q_0) \\
              &\approx y_0 + J (q - q_0)
\end{align}$
<br>
<br>
Here a Thikonov regularized iterative least squares solver is used. <br>

$\begin{align}
q^* &= \underset{q}{\text{argmin}} \ \lVert \phi(q) - y^* \rVert^2_C + \lVert q_0 - q \rVert^2_W &  \\
&= \underset{q}{\text{argmin}} \ (\phi(q) - y^*)^\top C (\phi(q) - y^*) + (q_0 - q)^\top W (q_0 - q) & \text{(Thikonov regularization)} \\
&\approx \underset{q}{\text{argmin}} \ (y_0 + J (q - q_0) - y^*)^\top C (y_0 + J (q - q_0) - y^*) + (q_0 - q)^\top W (q_0 - q) &
\end{align}$ <br>

$\begin{align}
\underset{q}{\text{argmin}} \ L(q) &\Rightarrow \frac{\partial \ L(q)}{\partial q} = 0 & \text{(The gradient vanishes at a minimum)}
\end{align}$ <br>

$\begin{align}
\frac{\partial \ L(q)}{\partial q} &= \frac{\partial}{\partial q}
[y_0 + J (q - q_0) - y^*]^\top \ C \ [y_0 + J (q - q_0) - y^*]
+ (q_0 - q)^\top W (q_0 - q) \\
%
&= \frac{\partial}{\partial q}
[J (q - q_0) + (y_0 - y^*)]^\top \ C \ [J (q - q_0) + (y_0 - y^*)]
+ (q_0 - q)^\top W (q_0 - q) \\
%
&= \frac{\partial}{\partial q}
(q - q_0)^\top J^\top \ C \ J (q - q_0)
+ 2 (y_0 - y^*)^\top \ C \ J (q - q_0)
+ (q_0 - q)^\top W (q_0 - q) \\
%
&= 2(q - q_0)^\top J^\top \ C \ J
+  2 (y_0 - y^*)^\top \ C \ J
+  2 (q_0 - q)^\top W = 0
\end{align}$ <br>
<br>

$\begin{align}
2(q - q_0) J^\top \ C \ J  2 (q_0 - q)^\top W
&= -2 (y_0 - y^*)^\top \ C \ J \\
%
(J^\top \ C^\top \ J + W^\top) (q - q_0)
&= J^\top C^\top (y_0 - y^*) \\
%
q &= q_0 + (J^\top \ C^\top \ J + W^\top)^{-1} J^\top C^\top (y_0 - y^*)
\end{align}$ <br>
<br>
But since the whole thing is only a linear approximation (first order Taylor) the solutions should be computed incrementally in small steps: <br>
<br>
$q_{t+1} = q_t + \ ({J_t}^\top \ C^\top \ J_t + W^\top)^{-1} {J_t}^\top C^\top \epsilon \ (y_t - y^*)$ <br>
with some $\epsilon < 1$ and make sure to recompute $J_t$ and $y_t$ each time step. <br>

Solving the linear system for $q_{t+1}$ is faster than actually computing the inverse btw: <br>
$\underbrace{({J_t}^\top \ C^\top \ J_t + W^\top)}_{A} q_{t+1} = \underbrace{q_t + \ {J_t}^\top C^\top \epsilon \ (y_t - y^*)}_{b}$ <br>
$\Rightarrow A \ q_{t+1} = b$ <br>


<h4> $\rightarrow$ The great thing about this method is that only the link transformations and the Jacobian have to be specified!</h4>
But even the Jacobian could be approximated using finite differences. <br>
<br>
The Woodbury identity can be applied to make the effects of $W$ and $C$ more transparent:

$\begin{align}
q &= q_0 + (J^\top \ C^\top \ J + W^\top)^{-1} J^\top C^\top (y_0 - y^*) \\
  &= q_0 + W^{-1} J^\top (C^{-1} + J W^{-1} J^\top)^{-1} (y_0 - y^*)
\end{align}$

Choosing an invertible $C$ ensures the inverse can always be solved even if the target position is not even reachable. <br>
As $C$ approaches $\infty \ I$ and $W$ approaches $I$ the solution becomes the exact (left) pseudo inverse.

In [8]:
def Jacobian(q):
    J = np.zeros((2, len(q['angles'])))
    trafos = kinematicChain(q['angles'], q['lenghts'])
    eff = effectorPositions(trafos)
    for i in range(0, len(q['angles'])):
        d = eff[i + 1] - eff[i]
        J[:, i] = [-d[1], d[0]]
    return J

In [9]:
def inverseKinematic(q, endeffectorGoal, cVal = 50.0, wVal = 1.0):
    trafos = kinematicChain(q['angles'], q['lenghts'])
    eff = effectorPositions(trafos)

    diff = endeffectorGoal - eff[-1]
    dist = np.sqrt(np.dot(diff, diff))
    
    maxSpeed = 0.05
    speed = 0.4
    
    J = Jacobian(q)
    C = cVal * np.eye(2)
    W = wVal * np.eye(len(q['angles']))
    
    JtCtJWt_inv = np.linalg.inv(np.dot(np.dot(J.T, C.T), J) + W.T)
    q['angles'] = q['angles'] + np.dot(JtCtJWt_inv, (diff / dist) * min(maxSpeed, speed * dist))
    
    #A = np.dot(np.dot(J.T, C.T), J) + W.T
    #epsilon = min(maxSpeed, speed * dist)
    #b = q['angles'] + np.dot(J.T, np.dot(C.T, (diff / dist) * epsilon))
    #q['angles'] = np.linalg.solve(A, b)
    
    return q

In [10]:
import IPython.display as disp
from ipywidgets import widgets

js = '''
<h1> Demo: Inverse Kinematic </h1>
Use the mouse to specify the target position.

<script src="http://d3js.org/d3.v3.min.js" charset="utf-8"></script>
<script type="text/javascript">
// Calls a python function which return value is passed to the response function.
// The repsonse has to a string be in the JSON format.
// TODO: put this into a library file
function execute(functionCall, repsonseFunction) {
    // FIXME: the first call fails here
    var responseCallback = function(data) {
        // TODO: check if field exits at all
        var dataString = data.content['data']['text/plain']
        repsonseFunction(JSON.parse(dataString.slice(1, -1)));
    }
    IPython.notebook.kernel.execute(functionCall,
        {'iopub' : { 'output' : responseCallback } }, { silent:false });
}

function dictToArray(dict) {
    return Object.keys(dict).map(function (key) { return dict[key]; });
}
</script>


<script type="text/javascript">function svgClick(x) {}</script>

<svg id="canvas" onmousemove="svgClick(evt)" width="800" height="600">
<rect id="bg" width="100%" height="100%" fill="#808080"/>
</svg>

<script type="text/javascript">
// no scrolling please
//require("notebook/js/outputarea").OutputArea.prototype._should_scroll = function(lines) { return false; }

var svg = d3.select("#canvas");
bbox = svg.node().getBBox();

function toPixelX(x) { return bbox.x + bbox.width  * (x * 0.5 + 0.5); }
function toPixelY(y) { return bbox.y + bbox.height * (y * 0.5 + 0.5); }
function toRelX(x)   { return ((x - bbox.x) / bbox.width ) * 2.0 - 1.0; }
function toRelY(y)   { return ((y - bbox.y) / bbox.height) * 2.0 - 1.0; }

fps = 20.0
ms = 1000.0 / fps

running = true; // pause if already close enough to goal
effectors = [];
convergence = 0
function requestEffectorsResponse(eff) {
    lastEffectors = effectors;
    effectors = dictToArray(eff.data);
    render(effectors);
    
    // stop if nothing intersting is happening -> save some resources
    if(lastEffectors.length > 0) {
        d = 0.0;
        for(i = 0; i < effectors.length; ++i) {
            dx = effectors[i][0] - lastEffectors[i][0];
            dy = effectors[i][1] - lastEffectors[i][1];
            d = dx*dx + dy*dy;
        }
        
        if(d < 0.000001) {
            convergence++;
        } else {
            convergence = 0;
        }
        
        if(convergence > 10) {
            running = false;
        }
    }
}

var linkFunction = d3.svg.line()
    .x(function(d) { return toPixelX(d[0]); })
    .y(function(d) { return toPixelY(d[1]); })
    .interpolate("basis");
    
function updateResponse(response) {
    execute("cb.requestEffectors()", requestEffectorsResponse);
}

function render(nodePositions) {
    links = svg.selectAll("path#links")
        .data([0]);

    links
        .enter()
        .append("path")
        .attr("id", "links")
        .attr("d", linkFunction(nodePositions))
        .attr("stroke", "blue")
        .attr("stroke-width", 8)
        .attr("fill", "none");
        
    links
        .attr("d", linkFunction(nodePositions))
        .attr("stroke", "blue")
        .attr("stroke-width", 8)
        .attr("fill", "none");
        
    links.exit().remove();
        
    eff = svg.selectAll("circle#effector")
        .data(nodePositions);
  
    eff.enter()
        .append("circle")
        .attr("id", "effector")
        .attr("fill", "#101010")
        .attr("cx", function(d) { return toPixelX(d[0]); })
        .attr("cy", function(d) { return toPixelY(d[1]); })
        .attr("r", 8);
        
    eff
        .attr("cx", function(d) { return toPixelX(d[0]); })
        .attr("cy", function(d) { return toPixelY(d[1]); });
        
    eff.exit()
        .remove();
}

if(window.updateInterval != undefined && window.updateInterval != 'undefined') {
    window.clearInterval(window.updateInterval);
}
window.updateInterval = setInterval(function() {
    if(running) {
        execute("cb.update()", updateResponse);
    }
}, ms);

function svgClick(evt) {
    running = true;
    x = evt.offsetX;
    y = evt.offsetY;
    execute("cb.mouse(" + x + ", " + y + ", " + toRelX(x) + ", " + toRelY(y) + ")", mouseResponse);
}

function mouseResponse(out) {
    goal = [out.x, out.y]
}
</script>
'''

def toJson(data):
    return pd.DataFrame(data).to_json(orient='split')

class pyCallbacks():
    def __init__(self, q):
        self.out = widgets.Output()
        self.q = q
        self.goal = [0.5, 0.5]
        self.cVal = 50.0
        self.wVal = 1.0
    
    def mouse(self, x, y, xrel, yrel):
        with self.out:
            self.goal = np.asarray([xrel, yrel])
            return ('{"x":%f,"y":%f}' % (xrel, yrel))
    
    def update(self):
        with self.out:
            if not self.goal is None:
                self.q = inverseKinematic(self.q, self.goal, self.cVal, self.wVal)
            return toJson([])
        
    def requestEffectors(self):
        with self.out:
            trafos = kinematicChain(self.q['angles'], self.q['lenghts'])
            eff = effectorPositions(trafos)
            return toJson(eff)

linkCount = 5
linkLength = 0.2
cb = pyCallbacks(q={'angles': np.ones(linkCount) / linkCount, 'lenghts': np.ones(linkCount) * linkLength})


def cValSlider(C_Value):
    cb.cVal = C_Value
widgets.interact(cValSlider, C_Value=widgets.FloatSlider(min=0.01, max=100.0, value=cb.cVal))

def wValSlider(W_Value):
    cb.wVal = W_Value
widgets.interact(wValSlider, W_Value=widgets.FloatSlider(min=0.01, max=10.0, value=cb.wVal))

disp.display(disp.HTML(js))
disp.display(cb.out)
# set the initial goal
disp.display(disp.HTML('<script type="text/javascript">goal = [%f, %f]</script>' % (cb.goal[0], cb.goal[1])))

Output()